In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import datetime

In [3]:
import time

In [4]:
pd.options.display.max_colwidth = 100

In [5]:
df = pd.read_csv('train_janatahack2.csv')

In [6]:
df.head()

,session_id,startTime,endTime,ProductList,gender
0,u16159,15/12/14 18:11,15/12/14 18:12,A00002/B00003/C00006/D28435/;A00002/B00003/C00006/D02554/;A00002/B00003/C00006/D28436/;A00002/B0...,female
1,u10253,16/12/14 14:35,16/12/14 14:41,A00001/B00009/C00031/D29404/;A00001/B00009/C00031/D02617/;A00001/B00009/C00031/D29407/;A00001/B0...,male
2,u19037,01/12/14 15:58,01/12/14 15:58,A00002/B00001/C00020/D16944/,female
3,u14556,23/11/14 2:57,23/11/14 3:00,A00002/B00004/C00018/D10284/;A00002/B00004/C00018/D10285/;A00002/B00004/C00018/D10286/,female
4,u24295,17/12/14 16:44,17/12/14 16:46,A00001/B00001/C00012/D30805/;A00001/B00001/C00012/D30806/,male


In [7]:
df.isnull().sum()

session_id     0
startTime      0
endTime        0
ProductList    0
gender         0
dtype: int64

In [8]:
df.tail()

,session_id,startTime,endTime,ProductList,gender
10495,u15442,18/11/14 7:39,18/11/14 7:42,A00002/B00002/C00007/D06407/;A00002/B00002/C00007/D06409/,female
10496,u17986,25/11/14 15:16,25/11/14 15:16,A00006/B00030/C00334/D11660/,female
10497,u22508,09/12/14 10:11,09/12/14 10:11,A00002/B00002/C00007/D18028/,female
10498,u17087,22/11/14 11:27,22/11/14 11:27,A00003/B00012/C00131/D09453/;A00003/B00012/C00073/D09454/,female
10499,u23137,19/12/14 3:11,19/12/14 3:19,A00002/B00001/C00010/D02309/;A00002/B00002/C00002/D32450/;A00002/B00001/C00059/D21254/;A00003/B0...,female


In [9]:
df.dtypes

session_id     object
startTime      object
endTime        object
ProductList    object
gender         object
dtype: object

In [10]:
df.shape

(10500, 5)

### Data Cleaning

In [14]:
def time2milliseconds(s):
    return time.mktime(datetime.datetime.strptime(s, "%d/%m/%Y  %H:%M").timetuple())

In [15]:
def dateFormat(x):
    new_x = ""
    lst = x.split("/")
    
    new_x = lst[0]+'/'
    new_x = new_x + lst[1]+'/'
    new_x = new_x + "20"+lst[2]
    
    return time2milliseconds(new_x)
    

In [20]:
dateFormat("20/01/13 11:13")

1358660580.0

In [21]:
df['startTimeMillisecond'] = df['startTime'].apply(lambda x:dateFormat(x))

In [22]:
df['endTimeMillisecond'] = df['endTime'].apply(lambda x:dateFormat(x))

In [40]:
def productListCleaning(x):
    lst = x.split("/")
    s = set()
    
    for e in lst:
        if ';' in e:
            s.add(e[1:])
        if e != "" and ';' not in e:
            s.add(e)
    
    return list(s)

In [41]:
df['cleanedProductList'] = df['ProductList'].apply(lambda x:productListCleaning(x))

In [42]:
df['cleanedProductList']

0                                                   [D28436, D02554, D28435, A00002, B00003, C00006, D28437]
1                           [D29404, D29411, B00009, D29410, D29407, C00031, D25444, D29418, D02617, A00001]
2                                                                           [D16944, C00020, B00001, A00002]
3                                                           [D10285, B00004, D10286, A00002, C00018, D10284]
4                                                                   [D30805, B00001, C00012, D30806, A00001]
                                                        ...                                                 
10495                                                               [D06409, C00007, A00002, B00002, D06407]
10496                                                                       [C00334, B00030, A00006, D11660]
10497                                                                       [B00002, C00007, D18028, A00002]
10498              